# Парсинг данных

In [11]:
import os
import csv
import json
import requests
import cfscrape
import traceback
from tqdm import tqdm
from time import sleep


PAUSE_TIME = 5 # Увеличиваем интервалы отправки запросов — борьба с капчей
CSV_NUMBER = 'test' # Постфикс названия создаваемой таблицы
CSV_PATH = os.path.normpath(os.path.join(os.getcwd(), 'csv')) # Создаём папку 'csv' для записи создаваемых таблиц
if not os.path.exists(CSV_PATH): # ...если такой не существовало ранее
    os.mkdir(CSV_PATH)
    print(f'Folder {CSV_PATH} has been created!')


# Словарь некоторых городов с номерами, объявления по которым можно искать на Циан
regions = {
    'msk': 1, # Москва
    'spb': 2, # Санкт–Петербург
    'ekb': 4743, # Екатеринбург
    'nsk': 4897, # Новосибирск
    'kzn': 4777, # Казань
    'nng': 4885, # Нижний Новгород
}

# Названия столбцов (header) будущей таблицы,
# которые связываются с отобранными признаками в create_table()
dataset = [
            [ 
                'region',
                'address',
                'price',
                'total_area',
                'kitchen_area', 
                'living_area',
                'rooms_count',
                'floor', 
                'floors_number',
                'build_date',
                'is_complete',
                'completion_year',
                'house_material',
                'parking',
                'decoration',
                'balcony',
                'longitude', 
                'latitude',
                'passenger_elevator', 
                'cargo_elevator', 
                'metro', 
                'metro_distance', 
                'metro_transport',
                'district',
                'is_apartments',
                'is_auction'
            ]
        ]


# Функция для обработки пропусков и булевых значений
def add_attr(attr):
    if isinstance(attr, bool):
        return int(attr)
        
    return attr if attr is not None else 'empty'


# Функция для создания экземпляра класса запросов
def get_session():
    # Передаваемые параметры для экземпляра (как получить — туториал ниже)

    headers = {
        'authority': 'www.cian.ru',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        # 'cookie': '_CIAN_GK=ba9cbcd7-318e-466f-acc9-76edbde6f3e8; _gcl_au=1.1.547104461.1710680659; tmr_lvid=57770ba711bf55b7a32e568f304f9f9f; tmr_lvidTS=1710680659331; login_mro_popup=1; sopr_utm=%7B%22utm_source%22%3A+%22direct%22%2C+%22utm_medium%22%3A+%22None%22%7D; uxfb_usertype=searcher; sopr_session=4690d323f46b4532; _gid=GA1.2.1804219744.1710680661; _ym_uid=1710680662970774452; _ym_d=1710680662; uxs_uid=df351200-e45e-11ee-85dd-e90cfbd1a20d; afUserId=f76d2a6f-9a78-48ed-8085-df3d3715ca3d-p; AF_SYNC=1710680661868; _ym_isad=2; _ym_visorc=b; adrdel=1; adrcid=AT0UT0wEPNDp1rk7PPeLAzg; session_region_id=1; session_main_town_region_id=1; __cf_bm=BbPuZP1Voe0Xuq68MmgPcpmXtot4vjcdiWJ1urjxOfw-1710681351-1.0.1.1-iLJsaBY6woS.xbe4OQIJPoqgWhSmKSUl1JipA4iphNKsAWiXU.WTC0SqaeNxX_1Lxz9uQGoa8WKXCzkHS53_2Q; _ga_3369S417EL=GS1.1.1710680661.1.1.1710681355.60.0.0; _ga=GA1.2.1802668360.1710680661; _dc_gtm_UA-30374201-1=1; tmr_detect=0%7C1710681357609',
        'referer': 'https://www.cian.ru/',
        'sec-ch-ua': '"Not(A:Brand";v="24", "Chromium";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7; Chromium GOST) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    }

    session = requests.Session()
    session.headers = headers
    return cfscrape.create_scraper(sess=session) # cfscrape — обход защиты от ботов Cloudflare


# Записываем всё в файл формата .csv
def recording_table():
    try:
        with open(os.path.join(CSV_PATH, f'data_{CSV_NUMBER}.csv'), mode='w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                for row in dataset:
                    writer.writerow(row)

        print(f'The dataset is written in file "data_{CSV_NUMBER}.csv"')
        return
            
    except Exception as error:
        print('Recording error!\n', traceback.format_exc())
        sleep(PAUSE_TIME)
        return
    

# Получаем формат json (питоновский dict) из нашего запроса Response        
def get_json(session, region_name, cur_page):
    # Параметры, которые отображаются в URL-запросе
#     https://www.cian.ru/cat.php/?deal_type=sale&engine_version=2&offer_type=flat&region=1&
#     room1=1&room2=1&room3=1&room4=1&room5=1&room6=1
    json_data = {
            'jsonQuery': {
                '_type': 'flatsale',
                'engine_version': {
                    'type': 'term',
                    'value': 2,
                },
                'region': {
                    'type': 'terms',
                    'value': [
                        regions[region_name],
                    ],
                },
                'room': {
                    'type': 'terms',
                    'value': [
                        1,
                        2,
                        3,
                        4,
                        5,
                        6,
                    ],
                },
                'page': {
                    'type': 'term',
                    'value': cur_page,
                },
                # Можно задавать дополнительные атрибуты фильтрации объявлений
                
                # 'price': {
                #     'type': 'range',
                #     'value': {
                #         'gte': min_price,
                #         'lte': max_price,
                #     },
                # },
            },
        }
    # Получаем запрос с заданными параметрами
    # Возвращаемое значение — bytes
    try:
        response = session.post('https://api.cian.ru/search-offers/v2/search-offers-desktop/',
                                json=json_data)

    except:
        return f'oops! Error {response.status_code}'

    # Получаем формат .json
    if (
        response.status_code != 204 and 
        response.headers["content-type"].strip().startswith("application/json")
    ):
        try:
            return response.json()
        except ValueError:
            return f'oops! ValueError!'


def create_table(region_name='nng', start_page=1, end_page=55, number_of_samples=10_000):
    # В Циан выдаются страницы в диапазоне [1, 54]
    if start_page < 1:
        start_page = 1
    if end_page > 55:
        end_page = 55
    
    session = get_session()

    cnt_samples = 0
    for cur_page in tqdm(range(start_page, end_page)): # tqdm — выводим прогресс выполнения цикла
        if cnt_samples >= number_of_samples:
            break
            
        data = get_json(session, region_name, cur_page)
        if data is None:
            print('oops! Captcha!')
            return
        if isinstance(data, str):
            continue
        
        # Отбираем из большого словаря то, что нам нужно (можно и больше — смотри data)
        for item in data['data']['offersSerialized']:
            cur_item = [
                    region_name,
                    add_attr(item["geo"]["userInput"]),
                    add_attr(item['bargainTerms']['priceRur']),
                    add_attr(item.get('totalArea')),
                    add_attr(item.get('kitchenArea')),
                    add_attr(item.get('livingArea')),
                    add_attr(item.get('roomsCount')),
                    add_attr(item.get('floorNumber')),
                    add_attr(item['building'].get('floorsCount')),
                    add_attr(item['building'].get('buildYear')),
                    add_attr(item['building']['deadline']['isComplete'] if item['building'].get('deadline') is not None else None),
                    add_attr(item['building']['deadline']['year'] if item['building'].get('deadline') is not None else None),
                    add_attr(item['building'].get('materialType')),
                    add_attr(item['building']['parking']['type'] if item['building'].get('parking') is not None else None),
                    add_attr(item.get('decoration')),
                    add_attr(item.get('balconiesCount')),
                    add_attr(item['geo']['coordinates']['lng']),
                    add_attr(item['geo']['coordinates']['lat']),
                    add_attr(item['building'].get('passengerLiftsCount')),
                    add_attr(item['building'].get('cargoLiftsCount')),
                    add_attr(','.join([str(x['name']) for x in item['geo']['undergrounds']if x is not None])),
                    add_attr(','.join([str(x['time']) for x in item['geo']['undergrounds'] if x is not None])),
                    add_attr(','.join([str(x['transportType']) for x in item['geo']['undergrounds'] if x is not None])),
                    add_attr(','.join([str(x['name']) for x in item['geo']['districts'] if x is not None])),
                    add_attr(item.get('isApartments')), 
                    add_attr(item.get('isAuction'))
                ]
            
            if cur_item not in dataset:
                dataset.append(cur_item)
                cnt_samples += 1
            else:
                continue

            if cnt_samples >= number_of_samples:
                break

        print(f'{cnt_samples} / {number_of_samples} | page: {cur_page}')
        sleep(PAUSE_TIME)
                
    recording_table()
    return


create_table()

  0%|                                                    | 0/54 [00:00<?, ?it/s]

28 / 10000 | page: 1


  2%|▊                                           | 1/54 [00:06<06:06,  6.91s/it]

56 / 10000 | page: 2


  4%|█▋                                          | 2/54 [00:13<05:44,  6.63s/it]

...

1370 / 10000 | page: 53


 98%|██████████████████████████████████████████▏| 53/54 [05:24<00:05,  5.77s/it]

1386 / 10000 | page: 54


100%|███████████████████████████████████████████| 54/54 [05:31<00:00,  6.14s/it]

The dataset is written in file "data_test.csv"
